In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [3]:
df = pd.read_csv("energydata_complete.csv")
df.shape

(19735, 29)

In [4]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [6]:
#drop given columns
df.drop(columns=["date", "lights"], inplace=True)

#normalizing dataset to a common scale
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
X = normalised_df.drop(columns=['Appliances'])
y = normalised_df['Appliances']

In [7]:
#dividing dataset into train-test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
#instantiating model
model = LinearRegression()
#fitting the model to the training dataset
model.fit(x_train, y_train)
#obtain predictions
predicted_values = model.predict(x_test)
#Mean Absolute Error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

In [9]:
#residual sum of squares
residual_sum = np.sum(np.square(y_test - predicted_values))
round(residual_sum, 2)

45.35

In [10]:
#root mean squared error
root_mean = np.sqrt(mean_squared_error(y_test, predicted_values))
round(root_mean, 3)

0.088

In [11]:
#coefficient of determination
coeff = r2_score(y_test, predicted_values)
round(coeff, 2)

0.15

In [14]:
def get_weights(model, feature, col_name):
    
    weights = pd.Series(model.coef_, feature.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Feature', col_name]
    weights_df[col_name].round(4)
    return weights_df


In [15]:
model_weights = get_weights(model, x_train, 'Model_Weight')
model_weights

,Feature,Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [18]:
#instantiating ridge regression model
ridge_reg = Ridge(alpha=0.4)
#fitting the model
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [19]:
#new root mean squared error
new_root_mean = np.sqrt(mean_squared_error(y_test, predicted_values))
round(new_root_mean, 3)

0.088

In [20]:
#instantiating lasso regression
lasso_reg = Lasso(alpha=0.001)

#fitting lasso rgression
lasso_reg.fit(x_train, y_train)


Lasso(alpha=0.001)

In [21]:
lasso_weights = get_weights(lasso_reg, x_train, 'Lasso_weight')
lasso_weights

,Feature,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [24]:
#lasso root mean squared error
lasso_root_mean = np.sqrt(mean_squared_error(y_test, predicted_values))
round(lasso_root_mean, 3)

0.088

In [33]:
#creating variables for my feature matrix and target vector
X1 = normalised_df['T2']
y1 = normalised_df['T6']


In [34]:
#dividing dataset into train-test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [35]:
#instantiating model
modelz = LinearRegression()
#fitting the model to the training dataset
modelz.fit(X1_train, y1_train)
#obtain predictions
predicted_valuez = modelz.predict(x_test)
#Mean Absolute Error
mae1 = mean_absolute_error(y1_test, predicted_valuez)
round(mae1, 2)

ValueError: Expected 2D array, got 1D array instead:
array=[17.79  18.7   16.89  ... 18.2   20.434 19.89 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [37]:
df["T6"].head()

0    7.026667
1    6.833333
2    6.560000
3    6.433333
4    6.366667
Name: T6, dtype: float64

In [38]:
###could not solve number 12 :(